In [1]:
#Question 3
#classification->supervised learning
#import packages
import pandas as pd #data processing and csv file
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    cohen_kappa_score, roc_auc_score, roc_curve, classification_report
)
from sklearn.model_selection import GridSearchCV #for tuning hyperparameter k in KNN
from sklearn.model_selection import StratifiedKFold #for KNN
from sklearn.preprocessing import StandardScaler
loan_df=pd.read_csv('loan.data.sample.csv')
loan_df.info()
loan_df.head()
#print(loan_df.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SK_ID_CURR           307511 non-null  int64  
 1   TARGET               307511 non-null  int64  
 2   NAME_CONTRACT_TYPE   307511 non-null  object 
 3   CODE_GENDER          307511 non-null  object 
 4   FLAG_OWN_CAR         307511 non-null  object 
 5   FLAG_OWN_REALTY      307511 non-null  object 
 6   CNT_CHILDREN         307511 non-null  int64  
 7   AMT_INCOME_TOTAL     307511 non-null  float64
 8   AMT_CREDIT           307511 non-null  float64
 9   AMT_ANNUITY          307499 non-null  float64
 10  AMT_GOODS_PRICE      307233 non-null  float64
 11  NAME_EDUCATION_TYPE  307511 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 28.2+ MB


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Secondary / secondary special
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Higher education
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Secondary / secondary special
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Secondary / secondary special
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Secondary / secondary special


In [3]:
#drop the missing values 
loan_df=loan_df.dropna()
loan_df.isna().sum() # check for missing values for surety

SK_ID_CURR             0
TARGET                 0
NAME_CONTRACT_TYPE     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
AMT_CREDIT             0
AMT_ANNUITY            0
AMT_GOODS_PRICE        0
NAME_EDUCATION_TYPE    0
dtype: int64

In [5]:
#DATA CLEANING AND PREPROCESSING

#encode categorical variables so they are numeric input for machine learning models
loan_df['CODE_GENDER'] = loan_df['CODE_GENDER'].map({'F': 0, 'M': 1})
loan_df['FLAG_OWN_CAR'] = loan_df['FLAG_OWN_CAR'].map({'N': 0, 'Y': 1})
loan_df['FLAG_OWN_REALTY'] = loan_df['FLAG_OWN_REALTY'].map({'N': 0, 'Y': 1})
loan_df = pd.get_dummies(loan_df, columns=['NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE'], drop_first=True)
# all bool columns to int
loan_df = loan_df.astype({col: 'int' for col in loan_df.columns if loan_df[col].dtype == 'bool'})
loan_df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special
0,100002,1,1.0,0,1,0,202500.0,406597.5,24700.5,351000.0,0,0,0,0,1
1,100003,0,0.0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0,1,0,0,0
2,100004,0,1.0,1,1,0,67500.0,135000.0,6750.0,135000.0,1,0,0,0,1
3,100006,0,0.0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,0,0,0,1
4,100007,0,1.0,0,1,0,121500.0,513000.0,21865.5,513000.0,0,0,0,0,1


In [9]:
#split feature and target variables
train_features=loan_df.drop('TARGET', axis=1)
train_target=loan_df.TARGET


In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.2, random_state=42)


X_train.isna().sum() #missing values in X_train set
#remove the missing values in the training and test set
X_train = X_train.dropna()
y_train = y_train[X_train.index] 
X_test = X_test.dropna()
y_test = y_test[X_test.index]

In [17]:
#implement standard scaler always after splitting the data into training and testing sets
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [19]:
#set K: no grid search
knn=KNeighborsClassifier(n_neighbors = 3,algorithm='auto', metric = 'minkowski', p = 2)
knn.fit(X_train_scaled,y_train)
y_pred = knn.predict(X_test)

knn_prob=knn.predict_proba(X_test_scaled)[:, 1]


#print results and performance metrics
accuracy=accuracy_score(y_test,y_pred)
precision=precision_score(y_test,y_pred)
recall=recall_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
c_kappa=cohen_kappa_score(y_test,y_pred)
roc_auc=roc_auc_score(y_test,knn_prob)

print("KNN Performance:")
print(f"Accuracy     : {accuracy:.4f}")
print(f"Precision    : {precision:.4f}")
print(f"Recall       : {recall:.4f}")
print(f"F1 Score     : {f1:.4f}")
print(f"Cohen's Kappa: {c_kappa:.4f}")
print(f"AUC-ROC      : {roc_auc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

C:\Users\Taya\anaconda7\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


KNN Performance:
Accuracy     : 0.9172
Precision    : 0.0000
Recall       : 0.0000
F1 Score     : 0.0000
Cohen's Kappa: 0.0000
AUC-ROC      : 0.5271

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56357
           1       0.00      0.00      0.00      5086

    accuracy                           0.92     61443
   macro avg       0.46      0.50      0.48     61443
weighted avg       0.84      0.92      0.88     61443



C:\Users\Taya\anaconda7\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Taya\anaconda7\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Taya\anaconda7\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Taya\anaconda7\Lib\site-packages\sklearn\m

In [22]:
#KNN MODEL
#balance the dataset make use of stratifiedkfold and finding optimal k value with grid search
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    cohen_kappa_score, roc_auc_score, classification_report
)
from scipy.stats import randint

stratkf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
knn_model = KNeighborsClassifier()
param_dist = {'n_neighbors': randint(1, 6)}  #Search for k between 1 and 5

random_search = RandomizedSearchCV(
    estimator=knn_model,
    param_distributions=param_dist,
    n_iter=5,
    scoring='accuracy',
    cv=stratkf,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_train_scaled, y_train)
best_k = random_search.best_params_['n_neighbors']
print(f"\nBest K found: {best_k}")

KNN_best_model = random_search.best_estimator_

knn_pred = KNN_best_model.predict(X_test_scaled)
knn_prob = KNN_best_model.predict_proba(X_test_scaled)[:, 1]

accuracy = accuracy_score(y_test, knn_pred)
precision = precision_score(y_test, knn_pred, zero_division=0)
recall = recall_score(y_test, knn_pred, zero_division=0)
f1 = f1_score(y_test, knn_pred, zero_division=0)
c_kappa = cohen_kappa_score(y_test, knn_pred)
roc_auc = roc_auc_score(y_test, knn_prob)

print("\nKNN Performance:")
print(f"Accuracy     : {accuracy:.4f}")
print(f"Precision    : {precision:.4f}")
print(f"Recall       : {recall:.4f}")
print(f"F1 Score     : {f1:.4f}")
print(f"Cohen's Kappa: {c_kappa:.4f}")
print(f"AUC-ROC      : {roc_auc:.4f}")

#Classification Report
print("\nClassification Report:")
print(classification_report(y_test, knn_pred, zero_division=0))


Fitting 2 folds for each of 5 candidates, totalling 10 fits

Best K found: 4

KNN Performance:
Accuracy     : 0.9148
Precision    : 0.1449
Recall       : 0.0061
F1 Score     : 0.0117
Cohen's Kappa: 0.0050
AUC-ROC      : 0.5337

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56357
           1       0.14      0.01      0.01      5086

    accuracy                           0.91     61443
   macro avg       0.53      0.50      0.48     61443
weighted avg       0.85      0.91      0.88     61443

